# Poincare Boundary Condition

Most of the tutorials use last closed flux surface (LCFS) as the boundary condition for 3D equilibrium. This tutorial will cover another way to define boundary condition using DESC.

In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../../"))

Import required stuff.

In [2]:
%matplotlib inline
import numpy as np

from desc.objectives import (
    ObjectiveFunction,
    ForceBalance,
    get_fixed_xsection_constraints,
    get_fixed_boundary_constraints,
)
from desc.examples import get
from desc.optimize import Optimizer
from desc.plotting import plot_comparison
from desc.geometry import PoincareSurface
from desc.equilibrium import Equilibrium

DESC version 0.10.4+217.g631e102e.dirty,using JAX backend, jax version=0.4.14, jaxlib version=0.4.14, dtype=float64
Using device: CPU, with 7.74 GB available memory


### Solving Poincare Boundary Condition from an existing equilibrium

To first show that Poincare BC gives the same result with LCFS, let's set our optimization problem by taking the $\zeta$=0 Poincare cross-section of Heliotron equilibrium.

In [ ]:
eq = get("HELIOTRON")
zeta = 0
eq_poin = eq.set_poincare_equilibrium(zeta=zeta)
plot_comparison(eqs=[eq,eq_poin],labels=['LCFS',f'Poincare BC Initial Guess']);

We can also choose $\zeta = \pi$ as the boundary condition and set it from an existing equilibrium.

In [ ]:
eq = get("HELIOTRON")
eq_poin = eq.set_poincare_equilibrium()

# plot_comparison(eqs=[eq,eq_poin],labels=['LCFS','Poincare BC Initial Guess']);

In [5]:
print(eq.get_poincare_xsection_at().L_basis)
# print(eq.Z_basis.L)
print(eq_poin.R_basis)
print(eq.R_basis)
# print(eq.xsection.isgiven)
# print(eq_poin.axis.R_n)
# print(eq_poin.get_axis().R_n)
# print(eq.get_axis().R_n)
# eq.print_summary()
# eq_poin.print_summary()

ZernikePolynomial at 0x7f7fb8f27b90 (L=24, M=12, N=0, NFP=1, sym=sin, spectral_indexing=fringe)
FourierZernikeBasis at 0x7f7fb865b650 (L=24, M=12, N=3, NFP=19, sym=cos, spectral_indexing=fringe)
FourierZernikeBasis at 0x7f7fbab9d950 (L=24, M=12, N=3, NFP=19, sym=cos, spectral_indexing=fringe)


[10.54318873  0.          0.          0.        ]
[1.04335180e+01 1.08086067e-01 1.56125363e-03 2.33675127e-05]


In [10]:
eq = eq_poin
ns = eq.axis.R_basis.modes[:, 2]
dim_f = ns.size
A = np.zeros((dim_f, eq.R_basis.num_modes))

for i, (l, m, n) in enumerate(eq.R_basis.modes):
    if m != 0:
        continue
    if (l // 2) % 2 == 0:
        j = np.argwhere(n == ns)
        A[j, i] = 1
    else:
        j = np.argwhere(n == ns)
        A[j, i] = -1
dot = np.dot(A, eq.R_lmn)
Ra_n = eq.Ra_n
print(dot)
print(Ra_n)
print(dot-Ra_n)

[10.54318873  0.          0.          0.        ]
[10.54318873  0.          0.          0.        ]
[0. 0. 0. 0.]


Now, we can solve the newly created 'eq_poin' as usual. The code will apply some checks to see if you are trying to solve a Poincare BC problem or LCFS. Then, add the required constraints and objectives before sending it to the optimizer.

In [3]:
eq = get("HELIOTRON")
eq_poin = eq.set_poincare_equilibrium()

constraints = get_fixed_xsection_constraints(eq=eq_poin)
objective = ObjectiveFunction(ForceBalance(eq_poin))
optimizer = Optimizer("lsq-exact")

eq_poin.solve(verbose=3, ftol=0, objective=objective, constraints=constraints, optimizer=optimizer,maxiter=1, xtol=0)
plot_comparison(eqs=[eq,eq_poin],labels=['LCFS',f'Poincare BC']);

Building objective: force
Precomputing transforms
Timer: Precomputing transforms = 2.35 sec
Timer: Objective build = 4.97 sec


AssertionError: 
Not equal to tolerance rtol=5e-14, atol=2e-14
Incompatible constraints detected, cannot satisfy constraint <desc.objectives.linear_objectives.AxisRSelfConsistency object at 0x7f7fb9f605d0>
Mismatched elements: 1 / 4 (25%)
Max absolute difference: 2.08721929e-14
Max relative difference: inf
 x: array([-2.664535e-15, -3.108624e-15,  1.554312e-14, -2.087219e-14])
 y: array([0, 0, 0, 0])

### Creating a new Poincare surface to fix 

We can also create a new surface object called "PoincareSurface" to set our optimization problem as LCFS.

In [ ]:
R_lmn = np.array([10, -1.2, -0.2, 0.1])
modes_R = np.array([[0, 0], [1, 1], [2, 0], [2, 2]])

Z_lmn = np.array([0.7, -0.04, -0.02])
modes_Z = np.array([[1, -1], [2, -2], [3, -3]])

L_lmn = np.array([.1])
modes_L = np.array([[1, -1]])

section = PoincareSurface(R_lmn=R_lmn, modes_R=modes_R,Z_lmn=Z_lmn, modes_Z=modes_Z,L_lmn=L_lmn, modes_L=modes_L)

eq_poincare = Equilibrium(
            xsection=section,
            pressure=0,
            iota=0,
            Psi=eq.Psi,  # flux (in Webers) within the last closed flux surface
            NFP=10,  # number of field periods
            L=5,  # radial spectral resolution
            M=5,  # poloidal spectral resolution
            N=4,  # toroidal spectral resolution
            L_grid=eq.L_grid,  # real space radial resolution, slightly oversampled
            M_grid=eq.M_grid,  # real space poloidal resolution, slightly oversampled
            N_grid=eq.N_grid,  # real space toroidal resolution
            sym=True,  # explicitly enforce stellarator symmetry
            spectral_indexing=eq._spectral_indexing,
        )

plot_comparison(eqs=[eq_poincare],labels=['Poincare Surface']);